In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import subprocess

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdmolops
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Cluster import Butina

# shut off warnings
from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')  

# Part 1: Define some helper functions for calculating similarity

In [6]:
# want to be able to incorporate new results into pandas dataframe so we can look at it in a csv file and within notebooks
def print_drug_dict(drugdict, smiles_list, col_list, names_list, merging_df):
    # some hardcoded stuff in here - not ideal
    ret = pd.DataFrame(columns = col_list)
    for key, item in drugdict.items():
        tan, index = item
        if names_list == None:
            ret = ret.append(pd.DataFrame([[key, tan, smiles_list[index], index]], columns = col_list))
        else:
            ret = ret.append(pd.DataFrame([[key, tan, smiles_list[index], names_list[index]]], columns = col_list))
    
    ret = ret.merge(merging_df, left_on = 'SMILES', right_on = 'SMILES')
    return(ret)

# for every molecule, get similarity to closest antibiotic
def get_lowest_tanimoto_from_drug_set(new_set, abx_fps, smiles_list, col_list, merging_df, names_list = None):
    mols = [Chem.MolFromSmiles(x) for x in new_set]
    fps = [Chem.RDKFingerprint(x) for x in mols] # we already know the new_set smis are all valid mols since they went thru chemprop

    best_similarity = {}
    index = 0
   
    for mol in fps:
        set_index = 0
        curr_highest_sim = 0
        curr_highest_drug = 0
        for abx in abx_fps:
            sim = DataStructs.FingerprintSimilarity(mol,abx)
            if sim > curr_highest_sim:
                curr_highest_sim = sim
                curr_highest_drug = set_index
            set_index = set_index + 1
        info_to_include = [curr_highest_sim, curr_highest_drug]
        best_similarity[new_set[index]] = info_to_include
        
        index = index + 1
    
    ret = print_drug_dict(best_similarity, smiles_list, col_list, names_list, merging_df)
    return(ret)

# preprocess abx + compute sims against
def compute_tanimoto_against_abx(smis, merging_df):
    abx = pd.read_csv('../data/chembl_abx.csv', sep = ';') # set of anything labelled 'Antibiotics' from ChEMBL
    not_nans = [type(smi) != float for smi in list(abx['Name'])]
    abx = abx[not_nans]
    not_nans = [type(smi) != float for smi in list(abx['Smiles'])]
    abx = abx[not_nans]

    names = list(abx['Name'])
    smiles = list(abx['Smiles'])
    mols = [Chem.MolFromSmiles(x) for x in smiles]
    fps = [Chem.RDKFingerprint(x) for x in mols]
    col_list = ['SMILES', 'tanimoto similarity to closest abx', 'closest abx smiles', 'closest abx name']
    print(len(smiles))
    print(len(names))
    print(len(mols))
    print(len(fps)) # NOTE that this code does not work if the fps length not equal to names/smiles 
    gen_mols = get_lowest_tanimoto_from_drug_set(smis, fps, smiles, col_list, merging_df, names)    
    return(gen_mols)

# preprocess training set + compute sims against
def compute_tanimoto_against_training_set(smis, merging_df):
    df = pd.read_csv('../out/data_prep_for_ml_fullset.csv')
    not_nans = [type(smi) != float for smi in list(df['SMILES'])]
    df = df[not_nans]

    smiles = list(df['SMILES'])
    names = list(df['Name'])
    mols = [Chem.MolFromSmiles(x) for x in smiles]
    fps = [Chem.RDKFingerprint(x) for x in mols if x is not None]
    print(len(smiles))
    print(len(names))
    print(len(mols))
    print(len(fps)) # NOTE that this code does not work if the fps length not equal to names/smiles 
    col_list = ['SMILES', 'tanimoto similarity to closest train set', 'closest train set smiles', 'closest train set name']
    gen_mols = get_lowest_tanimoto_from_drug_set(smis, fps, smiles, col_list, merging_df, names)
    return(gen_mols)

# Part 2: Process predictions from model

In [5]:
df = pd.read_csv('../out/broad800k_model_predictions_08_20_2021.csv')
df = df[df['hit_kill'] > 0.2]
df['SMILES'] = df['smiles']
meta = pd.read_csv('../data/PublicStructures.txt', sep = '\t')
df = df.merge(meta, on = 'SMILES')

smis = list(df['SMILES'])
print("Computing tanimoto scores against abx...")
df = compute_tanimoto_against_abx(smis, df)
print("Computing tanimoto scores against training set...")
df = compute_tanimoto_against_training_set(smis, df)
df = df[df['tanimoto similarity to closest train set'] < 1]
df.to_csv('../out/broad800k_model_predictions_kill02_08_20_2021.csv', index = False)
df

Computing tanimoto scores against abx...
Computing tanimoto scores against training set...


,SMILES,tanimoto similarity to closest train set,closest train set smiles,closest train set name,tanimoto similarity to closest abx,closest abx smiles,closest abx name,smiles,hit_inh,hit_inh_epi_unc,hit_kill,hit_kill_epi_unc,Name,PROJECT_CODE,PROJECT_NAME,CXSMILES,STEREO_COMMENTS
0,C[N+](C)(CCOc1ccccc1)Cc1ccccc1.OC(=O)c1cc2cccc...,0.435644,CCN1CCC[C@H]1CNC(=O)c1c(OC)ccc(Br)c1OC,remoxipride,0.319273,CCCCCCCCOc1ccc(C(=O)N[C@H]2C[C@@H](O)[C@@H](O)...,CILOFUNGIN,C[N+](C)(CCOc1ccccc1)Cc1ccccc1.OC(=O)c1cc2cccc...,0.089759,0.003944,0.234748,0.022272,BRD-M39654024,2001,General HTS Sets,OC(=O)c1cc2ccccc2c(Br)c1O.C[N+](C)(CCOc1ccccc1...,as drawn
1,CN(C)c1ccc(cc1)-c1nc2ccc3ccccc3c2c2c1CCC2,0.595642,CC[C@H](NC(=O)c1c(C)c(nc2ccccc12)-c1ccccc1)c1c...,SB-222200,0.476708,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(=O)c(c5o...,RIFALAZIL,CN(C)c1ccc(cc1)-c1nc2ccc3ccccc3c2c2c1CCC2,0.032539,0.000854,0.272220,0.030580,BRD-K92834382,2001,General HTS Sets,CN(C)c1ccc(cc1)-c1nc2ccc3ccccc3c2c2CCCc12,as drawn
2,O[C@@H](CNCc1ccccc1)Cn1c2ccc(cc2c2cc(ccc12)Cl)Cl,0.636771,CN(C)C[C@@H](O)Cn1c2ccc(Br)cc2c2cc(Br)ccc12 |&...,wiskostatin,0.350000,CC(C)Cn1cnc2c(N)nc3ccccc3c21,IMIQUIMOD,O[C@@H](CNCc1ccccc1)Cn1c2ccc(cc2c2cc(ccc12)Cl)Cl,0.069170,0.002910,0.236441,0.032415,BRD-A79555596,2001,General HTS Sets,O[C@@H](CNCc1ccccc1)Cn1c2ccc(Cl)cc2c2cc(Cl)ccc...,as drawn
3,Clc1ccc(cc1)-c1cn2c(nc3ccccc32)n1CCCNC1CCCCC1,0.492949,COc1cc2CC(C)(C)n3c(cc4nc5ccccc5nc34)-c2cc1OC,YM-90709,0.457711,CC[C@H]1OC(=O)[C@H](C)C(=O)[C@H](C)[C@@H](O[C@...,TELITHROMYCIN,Clc1ccc(cc1)-c1cn2c(nc3ccccc32)n1CCCNC1CCCCC1,0.046292,0.002590,0.283152,0.045922,BRD-K25973121,2001,General HTS Sets,Clc1ccc(cc1)-c1cn2c3ccccc3nc2n1CCCNC1CCCCC1,as drawn
4,O[C@@H](CNC1CCCCC1)CN1c2ccccc2Sc2ccccc21,0.806202,CN(C)CCCN1c2ccccc2Sc2ccccc12,promazine,0.317733,CN(C)[C@@H]1C(O)=C(C(=O)NCN[C@@H](C(=O)N[C@@H]...,PENIMOCYCLINE,O[C@@H](CNC1CCCCC1)CN1c2ccccc2Sc2ccccc21,0.042517,0.001975,0.202662,0.025442,BRD-A33120153,2001,General HTS Sets,"O[C@@H](CNC1CCCCC1)CN1c2ccccc2Sc2ccccc12 |&1:1,r|",as drawn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,ONC(=O)CCCCCCC(=O)N/N=C/C[P+](c1ccccc1)(c1cccc...,0.451411,CC1=C(C)C(=O)C(CCCCCCCCCC[P+](c2ccccc2)(c2cccc...,visomitin,0.205832,CC1=C\[C@@H](O)CC(=O)Cc2nc(co2)C(=O)N2CCC=C2C(...,VIRGINIAMYCIN,ONC(=O)CCCCCCC(=O)N/N=C/C[P+](c1ccccc1)(c1cccc...,0.152810,0.068002,0.231551,0.098328,BRD-K03937843,2001,General HTS Sets,ONC(=O)CCCCCCC(=O)N\N=C\C[P+](c1ccccc1)(c1cccc...,as drawn
513,C/C(=N\NC(=O)CCCCCCC(=O)Nc1ccccc1O)C[P+](c1ccc...,0.426415,CCCCCCCCCCC(C)(C)C(=O)Nc1c(OC)cc(OC)cc1OC,CI-976,0.349180,Cl.Cl.Cl.Cl.N=C(N)NCCCCC(=O)Nc1cc(C(F)(F)F)cc(...,BRILACIDIN TETRAHYDROCHLORIDE,C/C(=N\NC(=O)CCCCCCC(=O)Nc1ccccc1O)C[P+](c1ccc...,0.219038,0.067046,0.360840,0.107617,BRD-K95293872,2001,General HTS Sets,C\C(C[P+](c1ccccc1)(c1ccccc1)c1ccccc1)=N/NC(=O...,as drawn
514,CCCCCCc1nc(cc2c1[Si](O[C@H]2c1ccccc1)(C(C)C)C(...,0.630123,CC[C@H](C)[C@@H]1N[C@@H]2C=Cc3c(cc(nc3[C@H]2O)...,thiostrepton,0.615307,CN(C)[C@@H]1C(O)=C(C(=O)NCN[C@@H](C(=O)N[C@@H]...,PENIMOCYCLINE,CCCCCCc1nc(cc2c1[Si](O[C@H]2c1ccccc1)(C(C)C)C(...,0.177634,0.029326,0.217421,0.036573,BRD-K34709721,2001,General HTS Sets,CCCCCCc1nc(CCCc2cc3[C@@H](O[Si](C(C)C)(C(C)C)c...,as drawn
515,NCCCCNCCCN1c2ccccc2Sc2ccc(cc21)Cl,0.963359,CN(C)CCCN1c2ccccc2Sc2ccc(Cl)cc12,chlorpromazine,0.319296,COC(=O)[C@@H]1c2cc3c(c(O)c2[C@@H](O[C@@H]2O[C@...,NOGALAMYCIN,NCCCCNCCCN1c2ccccc2Sc2ccc(cc21)Cl,0.116063,0.006734,0.317389,0.024718,BRD-K51073125,5001,CMAP,NCCCCNCCCN1c2ccccc2Sc2ccc(Cl)cc12,as drawn
